In [2]:
#%run -i '../../scripts/utility_readblt.py' 'ee'

In [3]:
from utility_common import *
from utility_plotter import *
import ROOT
from IPython.display import clear_output
## add plotting option and load plotting parameter table
%matplotlib inline
#%qtconsole --style monokai
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

# 1. Read data and MC as dataframe

In [17]:
selection  = "ee"
nbjetcut   =  ">1"

pickledir  =  "/home/zchen/Documents/Analysis/workplace/data/pickle/{}/".format(selection)

cuts = GetSelectionCut(selection) + "& (nBJets{})".format(nbjetcut)

Data = LoadDataframe(pickledir + "data2016").query(cuts)
MCzz = LoadDataframe(pickledir + "mcdiboson").query(cuts)
MCdy = LoadDataframe(pickledir + "mcdy").query(cuts)
MCt  = LoadDataframe(pickledir + "mct").query(cuts)
MCtt = LoadDataframe(pickledir + "mctt").query(cuts)


In [12]:
#MCzz.eventWeight = 0.97*0.97*MCzz.eventWeight
#MCdy.eventWeight = 0.97*0.97*MCdy.eventWeight
#MCt .eventWeight = 0.97*0.97*MCt .eventWeight
#MCtt.eventWeight = 0.97*0.97*MCtt.eventWeight

In [18]:
MCsg = pd.concat([MCt,MCtt],ignore_index=True)

MCsgf = MCsg.query('(genCategory >= 16) ') # semilep and had
MCsg0 = MCsg.query('genCategory in [2,3,4,5,6,7,8,9,13,14,15]') #other lep
MCsg1 = MCsg.query('genCategory in [1]') #ee
MCsg2 = MCsg.query('genCategory in [10,11,12]') #etau

df_list = [MCzz,MCdy,MCsgf,MCsg0,MCsg1,MCsg2,Data]

# 2. Plot kinematics

In [19]:
pp = pd.read_csv("plotparameters.csv")
label_list = ['Diboson','V+Jets', r'$tt/tW \rightarrow l + had$',
              r'$tt/tW \rightarrow ll$ other',
              r'$tt/tW \rightarrow e + e$',
              r'$tt/tW \rightarrow e \tau$',
              'data']
color_list = ["#a32020", "#e0301e", "#eb8c00", "#49feec", "deepskyblue", "mediumpurple", "k"]
plotoutdir = GetPlotDir(selection, nbjetcut)
print(plotoutdir)

/home/zchen/Documents/Analysis/workplace/plot/ee/binned_nBJets/2b/


In [20]:
for index, row in pp.iterrows():
    
    v,a,b,step,xl = row["var"], row["lower"], row["upper"], row["step"], row["xlabel"]
    plotter = SelectionPlotter(v,a,b,step,df_list)
    plotter.settingPlot(xl,label_list,color_list)
    plotter.makePlot(plotoutdir)
    
    # print out process
    print("making plots: {0}/{1}".format(index,len(pp)))
    clear_output(wait=True)
    plt.close()

print('finish all making plots')

finish all making plots


# 3. Counting

In [7]:
ctr = SelectionCounter()

In [17]:
print("(nBJets{})".format(nbjetcut))

print("MCtt: {}".format(ctr.countSlt_ByTauDecay(MCtt,withweights=False)) )
print("MCtt width: {}".format(ctr.countSlt_ByTauDecay(MCtt,withweights=True)) )

print("MCt : {}".format(ctr.countSlt_ByTauDecay(MCt, withweights=False)) )
print("MCt width: {}".format(ctr.countSlt_ByTauDecay(MCt, withweights=True)) )

(nBJets==1)
MCtt: [127952, 0, 33, 702, 0, 0, 24, 0, 0, 20582, 5, 284, 4, 0, 0, 882, 0, 67, 0, 1, 2]
MCtt width: [108878.1390048604, 0, 28.220364895303547, 574.5829959107862, 0, 0, 19.319827933817354, 0, 0, 17245.72154139811, 4.497281851190328, 232.37740465880964, 3.4291204974055285, 0, 0, 707.5223480569742, 0, 50.68964706985206, 0, 0.904909313827753, 1.2242334102541208]
MCt : [908, 0, 0, 8, 0, 0, 0, 0, 0, 175, 0, 1, 0, 0, 0, 7, 0, 1, 0, 0, 0]
MCt width: [789.8441195429602, 0, 0, 6.86021474508047, 0, 0, 0, 0, 0, 142.27293665251767, 0, 0.9044389540076256, 0, 0, 0, 6.011371682476996, 0, 0.8612510952115059, 0, 0, 0]


In [18]:
ctr.countSlt_Scaled(df_list)

data:28217.0
TotalMC:28235.558486921065
------ MC break down ------
[  141.04290088  2306.18766275   155.09358938   129.17636781
 21957.17130891  3546.88665719]


In [24]:
for thisdf in [MCzz, MCdy, MCt, MCtt, Data]:
    n = np.sum(thisdf.eventWeight)
    nerr = np.sum(thisdf.eventWeight**2)**0.5
    print('{:.2f},{:.2f}'.format(n,nerr))

12.04,1.83
203.70,11.40
210.45,15.93
7873.63,37.03
8318.00,91.20


In [20]:
# print Generator information
f = ROOT.TFile("../../data/root/2016MC.root")
print("MCtt : {}".format(ctr.countGen_ByTauDecay(f,'ttbar_inclusive')) )
temp = np.array(ctr.countGen_ByTauDecay(f,'t_tw'))
temp+= np.array(ctr.countGen_ByTauDecay(f,'tbar_tw'))
print("MCt  : {}".format(list(temp)) )

MCtt : [1811409.0, 1811532.0, 3620281.0, 57057.0, 54227.0, 111125.0, 418126.0, 418126.0, 763147.0, 642646.0, 626108.0, 2350887.0, 643368.0, 627277.0, 2353727.0, 22670017.0, 22653517.0, 4016590.0, 3923504.0, 14718933.0, 70930278.0]
MCt  : [23105.0, 23040.0, 46342.0, 694.0, 717.0, 1420.0, 5260.0, 5260.0, 9727.0, 8199.0, 8029.0, 30054.0, 8163.0, 8006.0, 29970.0, 290101.0, 289467.0, 50864.0, 50050.0, 187515.0, 904067.0]
